# A Guided Tour of Ray Core: Multiprocessing Pool

[*Distributed multiprocessing.Pool*](https://docs.ray.io/en/latest/multiprocessing.html) make it easy to scale existing Python applications that use [`multiprocessing.Pool`](https://docs.python.org/3/library/multiprocessing.html) by leveraging *actors*.

---

First, let's start Ray…

In [1]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '192.168.84.193',
 'raylet_ip_address': '192.168.84.193',
 'redis_address': '192.168.84.193:6379',
 'object_store_address': '/tmp/ray/session_2021-12-27_18-57-49_243452_31818/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-27_18-57-49_243452_31818/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-27_18-57-49_243452_31818',
 'metrics_export_port': 62684,
 'node_id': 'b5f93ab517e64c767b10d610aea8163f3085291246052fdb6bfacd19'}

## Multiprocessing Pool example

The following is a simple Python function with a slight delay added (to make it behave like a more complex calculation)...

In [2]:
import time

def my_function (x):
    time.sleep(1)
    return x ** 2

In [3]:
%%time

my_function(11)

CPU times: user 51 ms, sys: 3.75 ms, total: 54.8 ms
Wall time: 1 s


121

First, let's try using this with Python's built-in [multiprocessing.Pool](https://docs.python.org/3/library/multiprocessing.html):

In [4]:
%%time

from multiprocessing import Pool

pool = Pool()

for result in pool.map(my_function, range(50)):
    print(result)

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
CPU times: user 349 ms, sys: 162 ms, total: 511 ms
Wall time: 8.1 s


Now we'll create a *Pool* using and distribute its tasks across a cluster (or across the available cores on a laptop):

In [5]:
%%time

from ray.util.multiprocessing import Pool

pool = Pool()

for result in pool.map(my_function, range(50)):
    print(result)

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
CPU times: user 530 ms, sys: 117 ms, total: 647 ms
Wall time: 8.15 s


## Pool will pickle the whole function to the Pool worker

In [7]:
import numpy as np
y = np.ones((5000, 10000))
def my_function2(x):
    return y.sum() + x.sum()

In [10]:
5000 * 10000 * y.itemsize * 1e-9

0.4

In [11]:
for result in pool.map(my_function2, [np.ones((200, 200))] * 10):
    print(result)

50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0


## but if your function try to fetch remote object, then there will no copy happens (in same node)
### actually this is made by Plasma object store

In [2]:
import numpy as np
y = np.ones((5000, 10000))
y_obj = ray.put(y)
def my_function3(x):
    y = ray.get(y_obj)
    return y.sum() + x.sum()

In [4]:
for result in pool.map(my_function3, [np.ones((200, 200))] * 10):
    print(result)

50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0
50040000.0


The distributed version has the trade-off of increased overhead, although now it can scale-out horizontally across a cluster. The benefits would be more pronounced with a more computationally expensive calculation.

Finally, shutdown Ray

In [13]:
ray.shutdown()